# Gradient Boosting From Scratch

Let's implement gradient boosting from scratch.

In [ ]:
from __future__ import print_function

import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from tensorflow.keras.datasets import boston_housing

np.random.seed(0)

plt.rcParams['figure.figsize'] = (8.0, 5.0)
plt.rcParams['axes.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 14

In [ ]:
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

In [ ]:
x_train.shape

## Exploration

Let explore the data before building a model. The goal is to predict the median value of owner-occupied homes in $1000s.


In [ ]:
# Create training/test dataframes for visualization/data exploration.
# Description of features: https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html
feature_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD','TAX', 'PTRATIO', 'B', 'LSTAT']
df_train = pd.DataFrame(x_train, columns=feature_names)
df_test = pd.DataFrame(x_test, columns=feature_names)

Exercise #1: What are the most predictive features? Determine correlation for each feature with the label. You may find the [corr](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html) function useful.

## Train Gradient Boosting model

Training Steps to build model an ensemble of $K$ estimators.
1. At $k=0$ build base model ,  $\hat{y}_{0}$: $\hat{y}_{0}=base\_predicted$
3. Compute residuals $r = \sum_{i=0}^n (y_{k,i} - \hat{y}_{k,i})$; $n: number\ train\ examples$
4. Train new model, fitting on residuals, $r$. We will call the predictions from this model $e_{k}\_predicted$
5. Update model predictions at step $k$ by adding residual to current predictions: $\hat{y}_{k} = \hat{y}_{k-1} + e_{k}\_predicted$
6. Repeat steps 2 - 5 `K` times.

In summary, the goal is to build K estimators that learn to predict the residuals from the prior model; thus we are learning to "correct" the
predictions up until this point.
<br>

$\hat{y}_{K} = base\_predicted\ +\ \sum_{j=1}^Ke_{j}\_predicted$

### Build base model

Exercise #2: Make an initial prediction using the `BaseModel` class -- configure the `predict` method to predict the training mean.

In [ ]:
class BaseModel(object):
    """Initial model that predicts mean of train set."""

    def __init__(self, y_train):
        self.train_mean = # TODO

    def predict(self, x):
        """Return train mean for every prediction."""
        return # TODO

def compute_residuals(label, pred):
    """Compute difference of labels and predictions.

    When using mean squared error loss function, the residual indicates the 
    negative gradient of the loss function in prediction space. Thus by fitting
    the residuals, we performing gradient descent in prediction space. See for
    more detail:

    https://explained.ai/gradient-boosting/L2-loss.html
    """
    return label - pred

def compute_rmse(x):
    return np.sqrt(np.mean(np.square(x)))

In [ ]:
# Build a base model that predicts the mean of the training set.
base_model = BaseModel(y_train)
test_pred = base_model.predict(x_test)
test_residuals = compute_residuals(y_test, test_pred)
compute_rmse(test_residuals)

Let's see how the base model performs on out test data. Let's visualize performance compared to the `LSTAT` feature.

In [ ]:
feature = df_test.LSTAT

# Pick a predictive feature for plotting.
plt.plot(feature, y_test, 'go', alpha=0.7, markersize=10)
plt.plot(feature, test_pred, label='initial prediction')

plt.xlabel('LSTAT', size=20)
plt.legend(prop={'size': 20});

There is definitely room for improvement. We can also look at the residuals:

In [ ]:
plt.plot(feature, test_residuals, 'bo', alpha=0.7, markersize=10)
plt.ylabel('residuals', size=20)
plt.xlabel('LSTAT', size=20)
plt.plot([feature.min(), feature.max()], [0, 0], 'b--', label='0 error');
plt.legend(prop={'size': 20});

### Train Boosting model

Returning back to boosting, let's use our very first base model as are initial prediction. We'll then perform subsequent boosting iterations to improve upon this model.

create_weak_model

In [ ]:
def create_weak_learner(**tree_params):
    """Initialize a Decision Tree model."""
    model = DecisionTreeRegressor(**tree_params)
    return model

Make initial prediction.

Exercise #3: Update the prediction on the training set (`train_pred`) and on the testing set (`test_pred`) using the weak learner that predicts the residuals.

In [ ]:
base_model = BaseModel(y_train)

In [ ]:
# Training parameters. 
tree_params = {
    'max_depth': 1,
    'criterion': 'mse',
    'random_state': 123
  }
N_ESTIMATORS = 50
BOOSTING_LR = 0.1

# Initial prediction, residuals.
train_pred = base_model.prediction(x_train)
test_pred = base_model.prediction(x_test)
train_residuals = compute_residuals(y_train, train_pred)
test_residuals = compute_residuals(y_test, test_pred)

# Boosting.
train_rmse, test_rmse = [], []
for _ in range(0, N_ESTIMATORS):
    train_rmse.append(compute_rmse(train_residuals))
    test_rmse.append(compute_rmse(test_residuals))
    # Train weak learner.
    model = create_weak_learner(**tree_params)
    model.fit(x_train, train_residuals)
    # Boosting magic happens here: add the residual prediction to correct
    # the prior model.
    grad_approx = # TODO
    train_pred += # TODO
    train_residuals = compute_residuals(y_train, train_pred)  
    
    # Keep track of residuals on validation set.
    grad_approx = # TODO
    test_pred += # TODO
    test_residuals = compute_residuals(y_test, test_pred)  

## Interpret results

Can you improve the model results?

In [ ]:
plt.figure()
plt.plot(train_rmse, label='train error')
plt.plot(test_rmse, label='test error')
plt.ylabel('rmse', size=20)
plt.xlabel('Boosting Iterations', size=20);
plt.legend()

## Let's visualize how the performance changes across iterations

In [ ]:
feature = df_test.LSTAT
ix = np.argsort(feature)

# Pick a predictive feature for plotting.
plt.plot(feature, y_test, 'go', alpha=0.7, markersize=10)
plt.plot(feature[ix], test_pred[ix], label='boosted prediction', linewidth=2)

plt.xlabel('feature', size=20)
plt.legend(prop={'size': 20});

Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License